In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
import requests
from config import ch_key

In [2]:
# PENN MUSEUM DATA
penn_csv = "penn.csv"
penn_df = pd.read_csv(penn_csv)
penn_df.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,10,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,emuIRN,curatorial_section,object_number,object_name,native_name,culture,provenience,material,period,date_made,...,technique,iconography,measurement_height,measurement_length,measurement_width,measurement_outside_diameter,measurement_tickness,measurement_unit,other_numbers,url
0,353827,American,40-16-1749,Vessel|Sherds,NaN,Cocle,Panama|Sitio Conte,Ceramic,NaN,AD 450 - 900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,B19 - 12,http://www.penn.museum/collections/object/353827
1,349877,American,40-16-949,Vessel|Pedestal|Sherds,NaN,Cocle,Panama|Sitio Conte,Ceramic,NaN,AD 450 - 900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,B19 - 50,http://www.penn.museum/collections/object/349877
2,349878,American,40-16-921,Carafe|Sherds,NaN,Cocle,Panama|Sitio Conte,Ceramic,NaN,AD 450 - 900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,B19 - 15,http://www.penn.museum/collections/object/349878
3,349879,American,40-16-1751,Vessel|Sherds,NaN,Cocle,Panama|Sitio Conte,Ceramic,NaN,AD 450 - 900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,B19 - 12,http://www.penn.museum/collections/object/349879
4,349880,American,40-16-1752,Handles|Sherds,NaN,Cocle,Panama|Sitio Conte,Ceramic,NaN,AD 450 - 900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,B19 - 12,http://www.penn.museum/collections/object/349880


In [ ]:
curl -X GET 'https://api.collection.cooperhewitt.org/rest/?method=cooperhewitt.departments.getInfo&access_token='

In [ ]:
# Save config information for COOPER HEWITT API
url = "https://api.collection.cooperhewitt.org/rest/?method=cooperhewitt.departments.getInfo&access_token=116d763e1cb9ebd1db800064804e842e&id=35347493"

# Build query URL
query_url = url + ch_key + "&id=35347493"



In [ ]:
penn_copy_df = penn_df[["emuIRN", "curatorial_section", "material", "object_name", "url"]].copy()
penn_copy_df.head()

In [2]:
# TATE MUSEUM DATA 
tate_url = "https://raw.githubusercontent.com/tategallery/collection/master/artist_data.csv"

tate_df = pd.read_csv(tate_url, delimiter = ",")
tate_df.head()

,id,name,gender,dates,yearOfBirth,yearOfDeath,placeOfBirth,placeOfDeath,url
0,10093,"Abakanowicz, Magdalena",Female,born 1930,1930.0,NaN,Polska,NaN,http://www.tate.org.uk/art/artists/magdalena-a...
1,0,"Abbey, Edwin Austin",Male,1852–1911,1852.0,1911.0,"Philadelphia, United States","London, United Kingdom",http://www.tate.org.uk/art/artists/edwin-austi...
2,2756,"Abbott, Berenice",Female,1898–1991,1898.0,1991.0,"Springfield, United States","Monson, United States",http://www.tate.org.uk/art/artists/berenice-ab...
3,1,"Abbott, Lemuel Francis",Male,1760–1803,1760.0,1803.0,"Leicestershire, United Kingdom","London, United Kingdom",http://www.tate.org.uk/art/artists/lemuel-fran...
4,622,"Abrahams, Ivor",Male,born 1935,1935.0,NaN,"Wigan, United Kingdom",NaN,http://www.tate.org.uk/art/artists/ivor-abraha...


In [5]:
#RENAME BIRTH/DEATH YEARS
new_tate_df = tate_df.rename(columns={'yearOfBirth':"birth_year",
                                          'yearOfDeath':'death_year'})
new_tate_df.head()

,id,name,gender,dates,birth_year,death_year,placeOfBirth,placeOfDeath,url
0,10093,"Abakanowicz, Magdalena",Female,born 1930,1930.0,NaN,Polska,NaN,http://www.tate.org.uk/art/artists/magdalena-a...
1,0,"Abbey, Edwin Austin",Male,1852–1911,1852.0,1911.0,"Philadelphia, United States","London, United Kingdom",http://www.tate.org.uk/art/artists/edwin-austi...
2,2756,"Abbott, Berenice",Female,1898–1991,1898.0,1991.0,"Springfield, United States","Monson, United States",http://www.tate.org.uk/art/artists/berenice-ab...
3,1,"Abbott, Lemuel Francis",Male,1760–1803,1760.0,1803.0,"Leicestershire, United Kingdom","London, United Kingdom",http://www.tate.org.uk/art/artists/lemuel-fran...
4,622,"Abrahams, Ivor",Male,born 1935,1935.0,NaN,"Wigan, United Kingdom",NaN,http://www.tate.org.uk/art/artists/ivor-abraha...


In [6]:
#separate column by delimiter FIRST NAME // LAST NAME
new_tate_df["first_name"] = new_tate_df["name"].str.split(",").str[1]

new_tate_df["last_name"] = new_tate_df["name"].str.split(",").str[0]
new_tate_df.head()

,id,name,gender,dates,birth_year,death_year,placeOfBirth,placeOfDeath,url,first_name,last_name
0,10093,"Abakanowicz, Magdalena",Female,born 1930,1930.0,NaN,Polska,NaN,http://www.tate.org.uk/art/artists/magdalena-a...,Magdalena,Abakanowicz
1,0,"Abbey, Edwin Austin",Male,1852–1911,1852.0,1911.0,"Philadelphia, United States","London, United Kingdom",http://www.tate.org.uk/art/artists/edwin-austi...,Edwin Austin,Abbey
2,2756,"Abbott, Berenice",Female,1898–1991,1898.0,1991.0,"Springfield, United States","Monson, United States",http://www.tate.org.uk/art/artists/berenice-ab...,Berenice,Abbott
3,1,"Abbott, Lemuel Francis",Male,1760–1803,1760.0,1803.0,"Leicestershire, United Kingdom","London, United Kingdom",http://www.tate.org.uk/art/artists/lemuel-fran...,Lemuel Francis,Abbott
4,622,"Abrahams, Ivor",Male,born 1935,1935.0,NaN,"Wigan, United Kingdom",NaN,http://www.tate.org.uk/art/artists/ivor-abraha...,Ivor,Abrahams


In [7]:
tate_data_df = new_tate_df.dropna(axis=0, how="any")

In [8]:
tate_data_df = tate_data_df[["first_name", "last_name", "gender", "birth_year","death_year", "url"]]
tate_data_df.head()

,first_name,last_name,gender,birth_year,death_year,url
1,Edwin Austin,Abbey,Male,1852.0,1911.0,http://www.tate.org.uk/art/artists/edwin-austi...
2,Berenice,Abbott,Female,1898.0,1991.0,http://www.tate.org.uk/art/artists/berenice-ab...
3,Lemuel Francis,Abbott,Male,1760.0,1803.0,http://www.tate.org.uk/art/artists/lemuel-fran...
10,Robert,Adam,Male,1728.0,1792.0,http://www.tate.org.uk/art/artists/robert-adam...
11,Harry William,Adams,Male,1868.0,1947.0,http://www.tate.org.uk/art/artists/harry-willi...


In [9]:
tate_data_df.dtypes

first_name     object
last_name      object
gender         object
birth_year    float64
death_year    float64
url            object
dtype: object

In [10]:
tate_data_df.loc[:,"birth_year"] = tate_data_df["birth_year"].astype("int64")

In [15]:
tate_data_df.loc[:,"death_year"] = tate_data_df["death_year"].astype("int64")

In [16]:
tate_data_df.count()

first_name    1372
last_name     1372
gender        1372
birth_year    1372
death_year    1372
url           1372
dtype: int64

In [17]:
tate_data_df.head()

,first_name,last_name,gender,birth_year,death_year,url
1,Edwin Austin,Abbey,Male,1852,1911,http://www.tate.org.uk/art/artists/edwin-austi...
2,Berenice,Abbott,Female,1898,1991,http://www.tate.org.uk/art/artists/berenice-ab...
3,Lemuel Francis,Abbott,Male,1760,1803,http://www.tate.org.uk/art/artists/lemuel-fran...
10,Robert,Adam,Male,1728,1792,http://www.tate.org.uk/art/artists/robert-adam...
11,Harry William,Adams,Male,1868,1947,http://www.tate.org.uk/art/artists/harry-willi...


In [18]:
tate_data_df.to_csv(r'Repos\date-homework\project2-etl\tate_data.csv')